#**Set up**

In [ ]:
!pip install trax==1.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached trax-1.4.1-py2.py3-none-any.whl (637 kB)
  Using cached funcsigs-1.0.2-py2.py3-none-any.whl (17 kB)
  Using cached tensorflow_text-2.12.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Using cached tensorflow-2.12.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (585.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 KB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 KB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 106.0 MB/s eta 0:00:00
  Attempting uninstal

In [ ]:
!pip list | grep trax

trax                          1.4.1


In [ ]:
!pip install simple-colors


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#**Import library**

In [ ]:
import json
import random
import numpy as np
from termcolor import colored
import trax
from trax import layers as tl
from trax.supervised import training
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Load data**



In [ ]:
DATA_FILE = 'data.json'
DATA_DIR = 'drive/MyDrive/52000820 - Deep learning/Cuoi Ki NLP/data'
DIALOGUE_DB = {}
VOCAB_FILE = 'en_32k.subword'
VOCAB_DIR = 'drive/MyDrive/52000820 - Deep learning/Cuoi Ki NLP/data/vocabs'

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def load_json(directory, file):
    with open(f'{directory}/{file}') as file:
        db = json.load(file)
    return db
DIALOGUE_DB = load_json(DATA_DIR, DATA_FILE)

In [ ]:
print(f'Số cuộc hội thoại: {len(DIALOGUE_DB)}')

Số cuộc hội thoại: 10438


Những người có hộp thoại đa miền (nhiều lĩnh vực) có "MUL" trong tên tệp của họ trong khi hộp thoại một miền có "SNG" hoặc "WOZ".

In [ ]:
# print 7 keys from the dataset to see the filenames
print(list(DIALOGUE_DB.keys())[0:7])

['SNG01856.json', 'SNG0129.json', 'MUL2168.json', 'SNG01445.json', 'MUL2105.json', 'PMUL1690.json', 'MUL2395.json']


Có 10.438 cuộc hội thoại, mỗi cuộc hội thoại nằm trong một tệp riêng.

Mỗi tệp cũng được tải vào một từ điển và mỗi tệp có ba khóa như sau:

In [ ]:
# get keys of the fifth file in the list above
print(DIALOGUE_DB['SNG0073.json'].keys())

dict_keys(['new_goal', 'goal', 'log'])


goal: chứa một số khóa liên quan đến mục tiêu của cuộc hội thoại

In [ ]:
DIALOGUE_DB['SNG0073.json']['goal']

{'taxi': {'info': {'leaveAt': '17:15',
   'destination': 'pizza hut fen ditton',
   'departure': "saint john's college"},
  'reqt': ['car type', 'phone'],
  'fail_info': {}},
 'police': {},
 'hospital': {},
 'hotel': {},
 'attraction': {},
 'train': {},
 'message': ["You want to book a <span class='emphasis'>taxi</span>. The taxi should go to <span class='emphasis'>pizza hut fen ditton</span> and should depart from <span class='emphasis'>saint john's college</span>",
  "The taxi should <span class='emphasis'>leave after 17:15</span>",
  "Make sure you get <span class='emphasis'>car type</span> and <span class='emphasis'>contact number</span>"],
 'restaurant': {}}

log: Nó là một danh sách các từ điển và mỗi thành phần của danh sách này cũng chứa một số mô tả.

In [ ]:
# get first element of the log list
DIALOGUE_DB['SNG0073.json']['log'][0]

{'text': "I would like a taxi from Saint John 's college to Pizza Hut Fen Ditton .",
 'metadata': {},
 'dialog_act': {'Taxi-Inform': [['Depart', 'saint johns college']]},
 'span_info': [['Taxi-Inform', 'Dest', 'pizza hut fen ditton', 11, 14],
  ['Taxi-Inform', 'Depart', "saint john 's college", 6, 9]],
 'turn_id': 0}

In [ ]:
print(' Person 1: ', DIALOGUE_DB['SNG0073.json']['log'][0]['text'])
print(' Person 2: ',DIALOGUE_DB['SNG0073.json']['log'][1]['text'])

 Person 1:  I would like a taxi from Saint John 's college to Pizza Hut Fen Ditton .
 Person 2:  What time do you want to leave and what time do you want to arrive by ?


In [ ]:
#trích xuất các cuộc hội thoại từ tệp của tập dữ liệu.
#Người 1: ' nếu 'x' là số chẵn hoặc ' Người 2: ' nếu 'x' là số lẻ.
'''
    Đầu vào:
        file (string): tên tệp được lưu dưới dạng json
        vd: SNG0073.json
        data_db (dict): cơ sở dữ liệu đối thoại
        vd: data.json
    Returns:
        gọi người nói tên SNG0073
        hàm trả về toàn bộ cuộc hội thoại của người SNG0073 và trợ lý ảo
'''
def get_conversation(file, data_db):
    result = ''
    len_msg_log = len(data_db[file]['log'])
    delimiter_1 = ' Person 1: '
    delimiter_2 = ' Person 2: '
    for i in range(len_msg_log):
        cur_log = data_db[file]['log'][i]
        if i % 2 == 0:
            result += delimiter_1
        else:
            result += delimiter_2
        result += cur_log['text']
    return result

In [ ]:
file = 'SNG01856.json'
conversation = get_conversation(file, DIALOGUE_DB)
print(conversation)

 Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel Person 2: Okay , do you have a specific area you want to stay in ? Person 1: no , i just need to make sure it 's cheap . oh , and i need parking Person 2: I found 1 cheap hotel for you that includes parking . Do you like me to book it ? Person 1: Yes , please . 6 people 3 nights starting on tuesday . Person 2: I am sorry but I was n't able to book that for you for Tuesday . Is there another day you would like to stay or perhaps a shorter stay ? Person 1: how about only 2 nights . Person 2: Booking was successful . 
 Reference number is : 7GAWK763 . Anything else I can do for you ? Person 1: No , that will be all . Good bye . Person 2: Thank you for using our services .


In [ ]:
import simple_colors

def print_conversation(conversation):
    delimiter_1 = 'Person 1: '
    delimiter_2 = 'Person 2: '
    split_list_d1 = conversation.split(delimiter_1)
    for sublist in split_list_d1[1:]:
        split_list_d2 = sublist.split(delimiter_2)
        print(simple_colors.blue(f'Person 1: {split_list_d2[0]}'))
        if len(split_list_d2) > 1:
            print(simple_colors.red(f'Person 2: {split_list_d2[1]}'))

print_conversation(conversation)

Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel 
Person 2: Okay , do you have a specific area you want to stay in ? 
Person 1: no , i just need to make sure it 's cheap . oh , and i need parking 
Person 2: I found 1 cheap hotel for you that includes parking . Do you like me to book it ? 
Person 1: Yes , please . 6 people 3 nights starting on tuesday . 
Person 2: I am sorry but I was n't able to book that for you for Tuesday . Is there another day you would like to stay or perhaps a shorter stay ? 
Person 1: how about only 2 nights . 
Person 2: Booking was successful . 
 Reference number is : 7GAWK763 . Anything else I can do for you ? 
Person 1: No , that will be all . Good bye . 
Person 2: Thank you for using our services .


Chúng ta sẽ chỉ sử dụng kết quả đầu ra của các lệnh gọi get_conversation để huấn luyện mô hình.

#**Xử lý dữ liệu phù hợp với inputs của mô hình Reformer**

Đầu vào mong muốn:

Person 1: Why am I so happy? Person 2: Because you are learning NLP Person 1: ... Person 2: ...*

- Person 1, Person 2 là dấu hiệu để mô hình biết ai là người đang nói để sinh văn bản phù hợp.

In [ ]:
#lấy tất cả các chuỗi hội thoại từ tất cả các tệp hội thoại và đặt chúng vào một danh sách.
all_files = DIALOGUE_DB.keys()
untokenized_data = []
for file in all_files:
    result = get_conversation(file, DIALOGUE_DB)
    untokenized_data.append(result)
print(untokenized_data[0])

 Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel Person 2: Okay , do you have a specific area you want to stay in ? Person 1: no , i just need to make sure it 's cheap . oh , and i need parking Person 2: I found 1 cheap hotel for you that includes parking . Do you like me to book it ? Person 1: Yes , please . 6 people 3 nights starting on tuesday . Person 2: I am sorry but I was n't able to book that for you for Tuesday . Is there another day you would like to stay or perhaps a shorter stay ? Person 1: how about only 2 nights . Person 2: Booking was successful . 
 Reference number is : 7GAWK763 . Anything else I can do for you ? Person 1: No , that will be all . Good bye . Person 2: Thank you for using our services .


In [ ]:
#Chia tập train = 95%, validation = 5%
random.shuffle(untokenized_data)
cut_off = int(len(untokenized_data) * .05)
train_data, eval_data = untokenized_data[:-cut_off], untokenized_data[-cut_off:]

print(f'Số lượng cuộc hội thoại: {len(untokenized_data)}')
print(f'Số lượng cuộc hội thoại trong tập train: {len(train_data)}')
print(f'Số lượng cuộc hội thoại trong tập validation: {len(eval_data)}')

Số lượng cuộc hội thoại: 10438
Số lượng cuộc hội thoại trong tập train: 9917
Số lượng cuộc hội thoại trong tập validation: 521


#**Tokenizing, batching with bucketing**

In [ ]:
def stream(data):
    while True:
        #chọn ngẫu nhiên data
        d = random.choice(data)
        # mang lại một tuple có hai giá trị giống hệt nhau
        # đầu vào của mô hình cũng sẽ là mục tiêu của chúng tôi trong quá trình đào tạo.
        yield (d, d)

In [ ]:
#Pipeline này được sử dụng để xử lý dữ liệu đầu vào của mô hình học sâu.
'''
    Phân chia các mẫu trong dữ liệu thành các "bucket" (thùng) có kích thước gần nhau,
    từ đó tạo ra các batch có kích thước khác nhau nhưng tối ưu.
    Ở đây, dữ liệu được phân chia thành các bucket với ngưỡng chiều dài là [128, 256, 512, 1024],
    và kích thước các batch được chọn tương ứng là [16, 8, 4, 2, 1].
'''
data_pipeline = trax.data.Serial(
    # Sắp xếp các mẫu trong dữ liệu theo thứ tự ngẫu nhiên.
    trax.data.Shuffle(),
    # Chuyển đổi các mẫu trong dữ liệu từ định dạng văn bản sang định dạng số (sử dụng từ điển từ vựng được cung cấp bởi vocab_dir và vocab_file).
    trax.data.Tokenize(vocab_dir=VOCAB_DIR,
                       vocab_file=VOCAB_FILE),

    # Lọc ra các mẫu trong dữ liệu có chiều dài vượt quá một giá trị ngưỡng (ở đây là 2048).
    trax.data.FilterByLength(2048),
    trax.data.BucketByLength(boundaries=[128, 256,  512, 1024],
                             batch_sizes=[16,    8,    4,   2, 1]),
    # Thêm trọng số vào các mẫu trong batch để đối phó với việc có những phần tử padding trong batch.
    #Các phần tử padding được đánh dấu bằng giá trị 0 và không được tính trong loss.
    trax.data.AddLossWeights(id_to_mask=0)
)

train_stream = data_pipeline(stream(train_data))
eval_stream = data_pipeline(stream(eval_data))

In [ ]:
def ReformerLM(vocab_size=33600, n_layers=6, mode='train', attention_type=tl.SelfAttention):
    model = tl.Serial(trax.models.reformer.ReformerLM(
                vocab_size = vocab_size,
                n_layers = n_layers,
                mode = mode,
                attention_type = attention_type
            ), tl.LogSoftmax()
    )
    model.init_from_file('/content/drive/MyDrive/52000820 - Deep learning/Cuoi Ki NLP/chatbot_model1.pkl.gz', weights_only=True)
    return model # tl.Serial(model, tl.LogSoftmax(),)

In [ ]:
#Hàm training_loop được sử dụng để định nghĩa một vòng lặp huấn luyện trên mô hình ReformerLM đã được tạo ra từ hàm ReformerLM.
'''
Đầu vào:
ReformerLM: mô hình được sử dụng để huấn luyện.
train_gen: dữ liệu huấn luyện.
eval_gen: dữ liệu đánh giá.
output_dir: đường dẫn để lưu trữ mô hình sau khi huấn luyện.
'''
def training_loop(ReformerLM, train_gen, eval_gen, output_dir = "./model/"):
  #learning rate schedule
    lr_schedule = trax.lr.warmup_and_rsqrt_decay(n_warmup_steps=1000, max_value=0.01)
    '''
    Tạo một TrainTask với các tham số:
    dữ liệu đã được gán nhãn (labeled_data=train_gen),
    hàm loss (loss_layer=tl.CrossEntropyLoss()),
    optimizer (optimizer=trax.optimizers.Adam(0.01)),
    learning rate schedule (lr_schedule=lr_schedule),
    và số bước huấn luyện trước khi in ra thông tin (n_steps_per_checkpoint=10).
    '''
    train_task = training.TrainTask(
        labeled_data=train_gen,
        loss_layer = tl.CrossEntropyLoss(),
        optimizer=trax.optimizers.Adam(0.01),
        lr_schedule=lr_schedule,
        n_steps_per_checkpoint=10
    )
    '''
    Tạo một EvalTask với các tham số:
    dữ liệu đã được gán nhãn (labeled_data=eval_gen)
    và các độ đo được sử dụng để đánh giá mô hình (metrics=[tl.CrossEntropyLoss(), tl.Accuracy()]).
    '''
    eval_task = training.EvalTask(labeled_data=eval_gen,
                                  metrics=[tl.CrossEntropyLoss(), tl.Accuracy()])
    '''
    Tạo một Loop với các tham số:
    mô hình (ReformerLM(mode='train')),
    TrainTask (train_task), EvalTask (eval_task),
    và đường dẫn đến thư mục để lưu trữ các checkpoint (output_dir=output_dir).
    '''
    loop = training.Loop(ReformerLM(mode='train'),
                         train_task,
                         eval_tasks=[eval_task],
                         output_dir=output_dir)
    return loop

In [ ]:

!rm -f model/model.pkl.gz
loop = training_loop(ReformerLM, train_stream, eval_stream)
#loop.load_checkpoint(directory='./model/', filename="model.pkl.gz")
loop.run(50)

/usr/local/lib/python3.9/dist-packages/jax/_src/xla_bridge.py:613: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:



Step      1: Total number of trainable weights: 70673640
Step      1: Ran 1 train steps in 95.53 secs
Step      1: train CrossEntropyLoss |  2.47367144


/usr/local/lib/python3.9/dist-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step      1: eval  CrossEntropyLoss |  2.32425284
Step      1: eval          Accuracy |  0.55584246

Step     10: Ran 9 train steps in 352.04 secs
Step     10: train CrossEntropyLoss |  2.17990279
Step     10: eval  CrossEntropyLoss |  2.07066226
Step     10: eval          Accuracy |  0.58304137

Step     20: Ran 10 train steps in 245.62 secs
Step     20: train CrossEntropyLoss |  2.14654207
Step     20: eval  CrossEntropyLoss |  1.98243856
Step     20: eval          Accuracy |  0.59179413

Step     30: Ran 10 train steps in 239.41 secs
Step     30: train CrossEntropyLoss |  1.90376878
Step     30: eval  CrossEntropyLoss |  1.64894140
Step     30: eval          Accuracy |  0.60908413

Step     40: Ran 10 train steps in 247.02 secs
Step     40: train CrossEntropyLoss |  1.77779830
Step     40: eval  CrossEntropyLoss |  1.70110977
Step     40: eval          Accuracy |  0.61294585

Step     50: Ran 10 train steps in 278.73 secs
Step     50: train CrossEntropyLoss |  1.71827245
Step     50

In [ ]:
def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]

def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)

In [ ]:
#Hàm ReformerLM_output_gen được sử dụng để tạo ra một generator cho phép dự đoán các từ tiếp theo của một câu bắt đầu với start_sentence
#bằng cách sử dụng mô hình ReformerLM đã được huấn luyện.
#temperature là tham số quyết định độ đa dạng của câu được tạo ra, càng cao thì kết quả sẽ càng khác nhau
def ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file, vocab_dir, temperature, tokenize=tokenize):
    input_tokens = tokenize(start_sentence, vocab_file, vocab_dir)
    input_tokens_with_batch = np.array(input_tokens)[None, :]
    output_gen = trax.supervised.decoding.autoregressive_sample_stream(
        ReformerLM,
        inputs=input_tokens_with_batch,
        temperature=temperature
    )
    return output_gen

In [ ]:
#tạo ra một tầng attention cho mạng Transformer
#giúp kiểm soát kích thước bộ nhớ và tốc độ tính toán trong quá trình dự đoán.
def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 120  # xác định số lượng tokens tối đa được giữ lại trong bộ nhớ tạm thời trong quá trình dự đoán.
    kwargs['predict_drop_len'] = 120  # chỉ định số lượng tokens tối đa được loại bỏ khi cập nhật bộ nhớ tạm thời.
    return tl.SelfAttention(*args, **kwargs)

model = ReformerLM(
    vocab_size=33600,
    n_layers=6,
    mode='predict',
    attention_type=attention,
)

In [ ]:
model.init_from_file('/content/model/model.pkl.gz',weights_only=True)#input_signature=shape11)
STARTING_STATE = model.state

In [ ]:
def generate_dialogue(ReformerLM, model_state, start_sentence, vocab_file, vocab_dir, max_len, temperature):
    delimiter_1 = 'Person 1: '
    delimiter_2 = 'Person 2: '
    sentence = ''
    counter = 0
    result = [tokenize(':', vocab_file=vocab_file, vocab_dir=vocab_dir)]
    ReformerLM.state = model_state
    output = ReformerLM_output_gen(model, start_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=temperature)
    for o in output:
        print(o.shape)
        result.append(o)
        sentence = detokenize(np.concatenate(result, axis=0), vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)
        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            return (f'{sentence}')
        elif sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            return (f'{sentence}')
        counter += 1
        if counter > max_len:
            break

In [ ]:
def concatenation_string(sample_sentence):
  replaced_chat = sample_sentence.replace('Person 1: ', '')
  replaced_chat = sample_sentence.replace('Person 2: ', '')
  replaced_chat = sample_sentence.replace(': ', '')
  return replaced_chat

In [ ]:
print(simple_colors.red('If you want to end the conversation, enter the number 1 '))
n = input()
prev_chat = ''
while n != '1':
  print(simple_colors.blue("Please enter your question: "))
  question = input()
  sample_sentence = "Person 1: " + prev_chat + question + " Person 2: " #nối chuỗi rồi xem nó như đầu vào để đưa vào dự đoán
  answer = generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)
  print(concatenation_string(answer))
  prev_chat = concatenation_string(sample_sentence + " " + answer)
  print(simple_colors.red('If you want to end the conversation, enter the number 1 '))
  n = input()

If you want to end the conversation, enter the number 1 
2
Please enter your question: 
I would like a taxi from Saint John's college to Pizza Hut Fen Ditton.
:When would you like to arrive by? 
If you want to end the conversation, enter the number 1 
2
Please enter your question: 
2
:I've got a grey Audi for you. The contact number is 07760322670. 
If you want to end the conversation, enter the number 1 
2
Please enter your question: 
nice, thanks
:Do you need any further assistance? 
If you want to end the conversation, enter the number 1 
2
Please enter your question: 
I'll call you when I need you
:No problem. Have a good day!Skoda! Person i i i want to go lovell i. i want to know. i need to know the postcode, and area. 
If you want to end the conversation, enter the number 1 
1
